In [ ]:
#Import the dpf & expamples library from ansys server
from ansys.dpf import core as dpf
from ansys.dpf.core import examples

datafile = examples.download_extrapolation_2d_result()

# Inbuilt datapoint integration with Gaussian Points derived from shape functions
data_integration_points = datafile["file_integrated"]

# Generate the mesh for 2D model
extrapolated_plate_model = dpf.Model(data_integration_points)
extrapolated_plate_mesh = extrapolated_plate_model.metadata.meshed_region

# Download the 3D crankshaft model from Ansys library
datafile_2 = examples.download_crankshaft()

# Inbuilt datapoint integration with Gaussian Points derived from shape functions
# data_integration_points = datafile["file_integrated"]

# Generate the mesh crankshaft model
crankshaft_model = dpf.Model(datafile_2)
crankshaft_mesh = crankshaft_model.metadata.meshed_region

# Plot& print Meshed Geometry details
extrapolated_plate_mesh.plot()
crankshaft_mesh.plot()
print(extrapolated_plate_mesh)
print(crankshaft_mesh)

In [ ]:
# Initializing displacement result operator to fetch result from the model
displacementop = dpf.operators.result.displacement()

# Collecting data integration Gauss points into field container through transition data source
data_sources_integration_points = dpf.DataSources(data_integration_points)
displacementop.inputs.data_sources.connect(data_sources_integration_points)
displacement_1 = displacementop.outputs.fields_container()

# Declaring displacement output separately in a field for scoping or iterating through all the
eqv = displacementop
field = eqv.outputs.fields_container()[0]
node_ids = field.scoping.ids

# Looping through all the nodes to obtain individual displacements in along each axis 
for node_id in node_ids[:100]:
    # Fetching each individual node by node ID
    node_coord = extrapolated_plate_model.metadata.meshed_region.nodes.node_by_id(node_id).coordinates
    print(f"Displacement on Node ID {node_id} : %8.5f, %8.5f, %8.5f" % tuple(node_coord))

#Evaluating normal stresses for 2D extrapolated model
stress_ep = extrapolated_plate_model.results.stress()
min_max_op = ops.min_max.min_max_fc(ops.math.norm_fc(stress_ep))
# Fetching minimum & maximum values of stresses 
print("Maximum Normal Stress for extrapolated frame is" , min_max_op.outputs.field_max().data[0],"Pa" )
print("Minimum Normal Stress for extrapolated frame is",min_max_op.outputs.field_min().data[0],"Pa")

# Mesh plot of the obtained displacement result
displacement_nodal_op = dpf.operators.averaging.elemental_mean_fc()
displacement_nodal_op.inputs.fields_container.connect(displacement_1)
extrapolated_plate_model.metadata.meshed_region.plot(displacement_nodal_op.outputs.fields_container())


# Initializing equivalent mechanical strain operator from dpf
strainop = dpf.operators.result.strain_eqv_as_mechanical()
data_sources_integration_points = dpf.DataSources(data_integration_points)
strainop.inputs.data_sources.connect(data_sources_integration_points)
strain_1 = strainop.outputs.fields_container()


strain_nodal_op = dpf.operators.averaging.elemental_mean_fc()
strain_nodal_op.inputs.fields_container.connect(strain_1)
extrapolated_plate_model.metadata.meshed_region.plot(strain_nodal_op.outputs.fields_container())


In [ ]:

# vm_op_1 = dpf.operators.averaging.nodal_difference()

# Declaring stress operators to calculate Von Misses stress

stress_op = dpf.operators.result.stress_von_mises()
# stress_op = dpf.operators.result.stress()
# Connecting the stress operators with crankshaft model
stress_op.inputs.connect(crankshaft_model)
stresses = stress_op.outputs.fields_container()
# Selecting nodal averaged method for computing & plotting all the Von-Mises stresses
stress_vm_op = dpf.operators.averaging.elemental_nodal_to_nodal_fc()
stress_vm_op.inputs.fields_container.connect(stresses)
#Generating final plot
crankshaft_mesh.plot(stress_vm_op.outputs.fields_container(),scalar_bar_args={"title": "Stress - Von Misses"})
# crankshaft_mesh.plot(stress_vm_op.outputs.fields_container(),scalar_bar_args={"title": "Normal Stress"})


#Finding minimum & maximum Von Mises stress
min_max_op = ops.min_max.min_max_fc(ops.math.norm_fc(stress_op))
print("Maximum Von Mises for crankshaft is" , min_max_op.outputs.field_max().data[0]/10**9,"GPa" )
print("Minimum Von Mises for crankshaft is",min_max_op.outputs.field_min().data[0]/1000,"kPa")


# Connecting the stress operators with crankshaft model for obtaining normal stress from available results
stress_op1 = dpf.operators.result.stress()
stress_op1.inputs.connect(crankshaft_model)
stresses_norm = stress_op1.outputs.fields_container()
# Applying Von Mises variant to normal stresses
vm_op = dpf.operators.invariant.von_mises_eqv()
vm_op.inputs.field.connect(stresses_norm)
von_mises = vm_op.outputs.field()
# Scooping through nodes in the output field
node_ids = von_mises.scoping.ids
# Initializing dataframe for scooping through all the nodes
df1=pd.DataFrame(["Null"])
# Iterating through all the nodes & recording the nodal displacements for each
i=1
for node_id in node_ids:
    # Fetch each individual node by node ID.
    node_coord = crankshaft_mesh.nodes.node_by_id(node_id).coordinates
    # Adding nodal displacements for all the nodes in the dataframe
    df1.loc[i]=["Deformation due to Von Mises stress on Node ID {node_id} : %8.5f, %8.5f, %8.5f" % tuple(node_coord)]
    i+=1

#Saving this data to excel in local
df1.to_excel("C:\\MISC\\IU_downloads\\Thesis\\nodal_disp_crankshaft.xlsx")
# stress_ep = extrapolated_plate_model.results.stress()
min_max_op = ops.min_max.min_max_fc(ops.math.norm_fc(stress_op))
print("Maximum Von Mises for crankshaft is" , min_max_op.outputs.field_max().data[0]/10**9,"GPa" )
print("Minimum Von Mises for crankshaft is",min_max_op.outputs.field_min().data[0]/1000,"kPa")
   


# for j in range(crankshaft_mesh.nodes.n_nodes):
#     # j=1
#     node_coord_1 = crankshaft_mesh.nodes.node_by_id(j+1).coordinates
#     print
#     (f"Crankshaft Displacement Node ID {j+1} : %8.5f, %8.5f, %8.5f" % tuple(node_coord_1))
    
    
    